In [20]:
import pandas as pd
from statsmodels.stats import power
from statsmodels.stats.proportion import (proportion_effectsize,
                                          proportions_ztest,
                                          proportion_confint)
from scipy.stats import mannwhitneyu
from scipy.stats import shapiro
from scipy.stats import norm
from scipy.stats import t

In [2]:
data = pd.read_csv('data/marketing_AB.csv')

## Проанализируйте структуру данных и проведите их предобработку:
#### Исследуйте структуру данных;

In [3]:
# Просмотр первых строк данных
print("Первые 5 строк данных")
print(data.head())

# Описание структуры данных
print("Структура данных")
print(data.info())

print("Статистика по числовым столбцам")
print(data.describe())

print("Проверка наличия дубликатов")

print(data.nunique())

Первые 5 строк данных
   Unnamed: 0  user id test group  converted  total ads most ads day  \
0           0  1069124         ad      False        130       Monday   
1           1  1119715         ad      False         93      Tuesday   
2           2  1144181         ad      False         21      Tuesday   
3           3  1435133         ad      False        355      Tuesday   
4           4  1015700         ad      False        276       Friday   

   most ads hour  
0             20  
1             22  
2             18  
3             10  
4             14  
Структура данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int

#### Преобразуйте столбцы к необходимым типам данных.

Заметим, что столбцы 'test group' и 'most ads day' имеют тип данных object, хотя они являются категориальными. Преобразуем их к типу category.


In [4]:
data['test group'] = data['test group'].astype('category')
data['most ads day'] =data['most ads day'].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Unnamed: 0     588101 non-null  int64   
 1   user id        588101 non-null  int64   
 2   test group     588101 non-null  category
 3   converted      588101 non-null  bool    
 4   total ads      588101 non-null  int64   
 5   most ads day   588101 non-null  category
 6   most ads hour  588101 non-null  int64   
dtypes: bool(1), category(2), int64(4)
memory usage: 19.6 MB


#### Проверьте данные на наличие пропусков и избавьтесь от них, если нужно.

In [5]:
# Проверка на пропуски
print("Количество пропусков в данных")
print(data.isnull().sum())



Количество пропусков в данных
Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64


**Вывод**: пропусков в данных нет

#### Проверьте, есть ли пользователи, которые в процессе A/Bтеста попали в обе группы. Если да, исключите пользователей, оказавшихся в обеих группах.

In [6]:
# Найти пользователей, которые присутствуют в обеих группах
overlap_users = data.groupby('user id')['test group'].nunique()
overlap_users = overlap_users[overlap_users > 1].index
print("Пользователи, которые попали в обе группы")
print(overlap_users)

Пользователи, которые попали в обе группы
Index([], dtype='int64', name='user id')


Вывод: пользователи, которые попали в обе группы, отсутствуют

## Сделайте первичный анализ результатов A/B-тестирования
#### Рассчитайте вспомогательные показатели для контрольной и тестовой групп:

- количество посещений сайта;
- суммарное количество совершённых покупок;

Сделайте промежуточные выводы по построенной таблице. Сопоставимо ли количество посещений обоих вариантов посадочной страницы? Можно ли говорить о сбалансированности выборок?

In [7]:
# Группировка данных по тестовой группе
grouped_data = data.groupby('test group', observed=False).agg(
    total_visits=('total ads', 'sum'),  # Суммарное количество посещений сайта (увиденной рекламы)
    total_purchases=('converted', 'sum'),  # Суммарное количество покупок (True = 1, False = 0)
    total_users=('user id', 'nunique')  # Количество уникальных пользователей
)

# Вывод результатов
print(grouped_data)



            total_visits  total_purchases  total_users
test group                                            
ad              14014701            14423       564577
psa               582481              420        23524


**Вывод**: Количество рекламы, увиденной из группы ad составляет 14014701, а из группы psa - 582481, суммарное количество покупок для группы ad - 14423 против 403 для psa. Если говорить о сбалансированности выборок, то можно сделать вывод, что выборки не сбалансированы, так как количество посещений сайта и суммарное количество покупок в группе ad значительно больше, чем в группе psa. Но для для корректного анализа необходимо рассчитать ключевые показатели для обеих групп - конверсия и среднее количество рекламы, увиденной пользователем.

#### В каждой из групп рассчитайте ключевые показатели:

- конверсия;
- среднее количество рекламы, увиденной пользователем.

Сделайте первичные выводы о результатах A/B-тестирования на основе показателей конверсии и среднего количества увиденной рекламы в каждой из групп. По какому(-им) показателю(-ям), на первый взгляд, вариант А эффективнее варианта B и наоборот?



In [8]:
# Расчет ключевых показателей

# Конверсия - количество покупок/количество рекламы, увиденной пользователем


grouped_data['conversion'] = ((grouped_data['total_purchases'] * 100)/ grouped_data['total_users']).round(2)


# Среднее количество рекламы, увиденной пользователем 

grouped_data['average_ads'] = grouped_data['total_visits'] / grouped_data['total_users']



print(grouped_data)




            total_visits  total_purchases  total_users  conversion  \
test group                                                           
ad              14014701            14423       564577        2.55   
psa               582481              420        23524        1.79   

            average_ads  
test group               
ad            24.823365  
psa           24.761138  


Вывод: группа "ad" показала значительно лучшие результаты по всем ключевым показателям: большему количеству посещений, покупок и более высокой конверсии. Это свидетельствует о том, что рекламная стратегия в группе "ad" была более успешной по сравнению с группой "psa".

In [9]:
daily_data = data.groupby(['most ads day','test group'], observed=False).agg({
    'user id':'count',
    'converted':'sum'
}).reset_index().rename(columns={'user id': 'users_count'})
daily_data['conversion']=daily_data['converted']*100/daily_data['users_count']
daily_data['conversion']=daily_data['conversion'].round(2)

daily_data.sort_values(by='most ads day', ascending=True)
conversion_piv = daily_data.groupby('test group', observed=False)['conversion'].agg(
    ['mean', 'median']
)
display(conversion_piv)

,mean,median
test group,,
ad,2.557143,2.46
psa,1.770000,1.63


## Проведите статистический анализ результатов A/B-тестирования
Сформулируйте статистические гипотезы, соответствующие поставленным бизнес-вопросам, и выберите статистический тест для их проверки.

Не забудьте проверить данные на нормальность там, где это необходимо.

С помощью аппарата статистических тестов определите:

Есть ли статистическая разница между конверсиями в группах А и B?

Есть ли статистическая разница между средними количествами увиденной рекламы в группах А и B?


**Нулевая гипотеза (H0)**: Конверсия пользователей, увидевших рекламу (группа "ad"), не отличается от конверсии пользователей, увидевших объявления государственной службы (группа "psa").  
 H0: p_(ad) = pₚₛₐ 

**Альтернативная гипотеза (H1)**: Конверсия пользователей, увидевших рекламу (группа "ad"), выше, чем у пользователей, увидевших объявления государственной службы (группа "psa")

Так как мы сравниваем две пропорции (конверсии) из независимых выборок (группы "ad" и "psa"), целесообразно использовать Z-тест для пропорций. Этот тест подходит, когда:

- Данные представляют собой бинарные исходы (покупка или нет).

- Выборки достаточно велики (обычно n > 30).

- Мы предполагаем, что данные распределены нормально.

In [10]:
# Сбалансируем выборки
p_control = conversion_piv.loc['psa','mean']/100
p_treatment = conversion_piv.loc['ad','mean']/100

effect_size = proportion_effectsize(p_control, p_treatment)

alpha = 0.05
beta = 0.2

size_sample = power.NormalIndPower().solve_power(effect_size=effect_size, alpha=alpha, power=1-beta, ratio=1)

print(f'Размер выборки: {size_sample:.0f}')


Размер выборки: 5319


In [11]:
# Сформируем выборки
control_sample = data[data['test group'] == 'psa'].sample(n=int(size_sample), random_state=0)
treatment_sample = data[data['test group'] == 'ad'].sample(n=int(size_sample), random_state=0)

# Объединим выборки
sample = pd.concat([control_sample, treatment_sample], axis=0)
sample

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
68544,68544,922723,psa,True,30,Tuesday,14
572632,572632,903370,psa,False,24,Saturday,11
579324,579324,901579,psa,False,2,Monday,7
72528,72528,923298,psa,False,54,Saturday,17
512269,512269,904742,psa,False,2,Thursday,13
...,...,...,...,...,...,...,...
334332,334332,1484523,ad,False,3,Friday,11
200429,200429,1650565,ad,False,29,Wednesday,19
526897,526897,1042502,ad,False,33,Monday,10
422945,422945,1083716,ad,False,9,Thursday,22


In [12]:
data_ = sample.groupby(['test group'], observed=False).agg({
    'user id':'count',
    'converted':'sum',
    'total ads':'sum'
}).reset_index().rename(columns={'user id': 'users_count'})
data_['mean_ads']=data_['total ads']/data_['users_count']
data_['mean_ads']=data_['mean_ads'].round(2)

data_['conversion']=data_['converted']*100/data_['users_count']
data_['conversion']=data_['conversion'].round(2)
data_

,test group,users_count,converted,total ads,mean_ads,conversion
0,ad,5318,139,130345,24.51,2.61
1,psa,5318,98,137833,25.92,1.84


In [13]:
count_conversions_ad=data_[data_['test group']=='ad']['converted'].sum()
count_conversions_psa=data_[data_['test group']=='psa']['converted'].sum()

total_ads_ad=data_[data_['test group']=='ad']['total ads'].sum()
total_ads_psa=data_[data_['test group']=='psa']['total ads'].sum()

total_observations_ad=data_[data_['test group']=='ad']['users_count'].sum()
total_observations_psa=data_[data_['test group']=='psa']['users_count'].sum()

count_purchase = [count_conversions_ad, count_conversions_psa]
count_ads=[total_ads_ad, total_ads_psa]  
count_observation = [total_observations_ad, total_observations_psa] 
z_stat, p_value = proportions_ztest(count_purchase, count_observation)

print(f'Z-статистика: {z_stat:.2f}')
print(f'p-value: {p_value:.4f}')

Z-статистика: 2.69
p-value: 0.0071


#### Вывод
Полученное значение p-value меньше уровня значимости 0.05, поэтому мы отвергаем нулевую гипотезу о равенстве конверсий пользователей в группах "ad" и "psa". Таким образом, можно сделать вывод о том, что реклама более эффективна, чем объявления государственной службы в плане конверсии пользователей в покупателей.

**Нулевая гипотеза (H0)**: Среднее количество рекламы, увиденной пользователем, не отличается в группах "ad" и "psa".

**Альтернативная гипотеза (H1)**: Среднее количество рекламы, увиденной пользователем, отличается в группах "ad" и "psa".

In [14]:
# Проверим данные на нормальность с помощью критерия Шапиро-Уилка

alpha = 0.05

shapiro_result_a = shapiro(data_[data_['test group']=='ad']['total ads'])
shapiro_result_b = shapiro(data_[data_['test group']=='psa']['total ads'])

print('p-value для группы ad:', shapiro_result_a.pvalue)
print('p-value для группы psa:', shapiro_result_b.pvalue)






# Используем U-статистику Манна-Уитни для проверки гипотезы о равенстве средних двух групп

alpha = 0.05

results = mannwhitneyu(
    x=data_[data_['test group']=='ad']['users_count'].sum(),
    y=data_[data_['test group']=='psa']['users_count'].sum(),
    alternative='two-sided'
)
print('p-value:', round(results.pvalue, 2))

p-value для группы ad: nan
p-value для группы psa: nan
p-value: 1.0


/var/folders/v8/yvjcp58143j87hsd8xmzy54w0000gn/T/ipykernel_89905/1794627864.py:5: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_result_a = shapiro(data_[data_['test group']=='ad']['total ads'])
/var/folders/v8/yvjcp58143j87hsd8xmzy54w0000gn/T/ipykernel_89905/1794627864.py:6: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_result_b = shapiro(data_[data_['test group']=='psa']['total ads'])


#### Вывод
Полученное значение p-value = 1 и больше уровня значимости 0.05, поэтому мы не можем отвергнуть нулевую гипотезу о равенстве средних количеств рекламы, увиденной пользователем, в группах "ad" и "psa". Таким образом, можно сделать вывод о том, что среднее количество рекламы, увиденной пользователем, не отличается в группах "ad" и "psa". Так же p-value при проверке на нормальность у групп ad и psa слишком мало (nan), что говорит о том, что данные не распределены нормально.

Подкрепите результаты статистических тестов, построив 95 % доверительные интервалы для:

- конверсий в каждой из групп;
- разницы конверсий в группах;
- среднего количества увиденной рекламы в группах А и B.

Проинтерпретируйте построенные доверительные интервалы для конверсий и среднего количества увиденной рекламы группах. Есть ли перекрытие между доверительными интервалами для групп? Если есть, то о чём это говорит?

In [16]:
# Расчет доверительных интервалов для конверсий в каждой из групп

# Разделим данные по группам
data_psa = data[data['test group']=='psa']
data_ad = data[data['test group']=='ad']

# Функция по расчету доверительного интервала для конверсии в группе
def calculate_conf_interval(x, n):
    alpha=0.05
    z = - norm.ppf(alpha / 2)
    eps = z * (x * (1 - x) / n) ** 0.5
    lower_ = x - eps 
    upper_ = x + eps 
    return lower_, upper_
    
# Расчет доверительного интервала конверсии для ad
lower_ad, upper_ad = calculate_conf_interval(data_ad['converted'].mean(), data_ad['user id'].count())
# Расчет доверительного интервала конверсии для psa
lower_psa, upper_psa = calculate_conf_interval(data_psa['converted'].mean(), data_psa['user id'].count())

print(f'Доверительный интервал для ad: [{lower_ad*100:.2f}, {upper_ad*100:.2f}]')
print(f'Доверительный интервал для psa: [{lower_psa*100:.2f}, {upper_psa*100:.2f}]')



Доверительный интервал для ad: [2.51, 2.60]
Доверительный интервал для psa: [1.62, 1.95]


Вывод: доверительные интервалы у конверсий в разных группах друг с другом не пересекаются, следовательно, конверсия в обеих группах не может совпадать.

In [19]:
# Расчет доверительного интервала для разницы конверсий в группах
def calculate_conf_interval_diff(x1, n1, x2, n2):
    alpha=0.05
    z = - norm.ppf(alpha / 2)
    eps = z * ((x1 * (1 - x1) / n1 + x2 * (1 - x2) / n2) ** 0.5)
    lower_ = (x2 - x1) - eps
    upper_ = (x2 - x1) + eps
    return lower_, upper_


# Расчет доверительного интервала для разницы конверсий в группах
lower_diff, upper_diff = calculate_conf_interval_diff(data_ad['converted'].mean(), data_ad['user id'].count(), data_psa['converted'].mean(), data_psa['user id'].count())

print(f'Доверительный интервал для разницы конверсий в группах: [{lower_diff*100:.2f}, {upper_diff*100:.2f}]')

Доверительный интервал для разницы конверсий в группах: [-0.94, -0.60]


Вывод: доверительный интервал для разницы конверсий отрицательный, что говорит о том, что конверсия в группе ad выше, чем в группе psa.

In [21]:
# Расчет доверительного интервала для среднего количества рекламы, увиденной пользователем

def calculate_conf_interval_mean(x_mean, x_std, n):
    alpha=0.05
    k = n - 1
    t_crit = -t.ppf(alpha/2, k)
    eps = t_crit * x_std/(n ** 0.5) 
    lower_ = x_mean - eps
    upper_ = x_mean + eps
    return lower_, upper_

# Расчет доверительного интервала для среднего количества рекламы, увиденной пользователем в группе ad
lower_ad, upper_ad = calculate_conf_interval_mean(data_ad['total ads'].mean(), data_ad['total ads'].std(), data_ad['user id'].count())
# Расчет доверительного интервала для среднего количества рекламы, увиденной пользователем в группе psa
lower_psa, upper_psa = calculate_conf_interval_mean(data_psa['total ads'].mean(), data_psa['total ads'].std(), data_psa['user id'].count())

print(f'Доверительный интервал для ad: [{lower_ad:.2f}, {upper_ad:.2f}]')
print(f'Доверительный интервал для psa: [{lower_psa:.2f}, {upper_psa:.2f}]')


Доверительный интервал для ad: [24.71, 24.94]
Доверительный интервал для psa: [24.21, 25.31]


Вывод: доверительные интервалы у среднего количества рекламы, увиденной пользователем, в разных группах друг с другом пересекаются, следовательно, среднее количество рекламы, увиденной пользователем, в обеих группах может совпадать.

Заключение: построенные доверительные интервалы подтверждают результаты статистических тестов. Конверсия в группе ad выше, чем в группе psa, следовательно, реклама более эффективна, чем объявления государственной службы